In [2]:
import os
import csv
import random
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

## Set `n_images` to a small number for debugging

In [3]:
n_images = 12273

## For reference: What is the image size?

## Fetch the weights of our trained ML encoder

In [ ]:
folder = Path("RESULTS")
kwargs = {"weights_only": True}

def get_weights(path):
    return torch.load(folder / path, **kwargs)[:n_images]

int_rep = get_weights("internal_rep.pt")
rgb_int_rep = get_weights("rgb_internal_rep.pt")
ir_int_rep = get_weights("ir_internal_rep.pt")

print("int_rep.shape", int_rep.shape)
print("rgb_int_rep.shape", rgb_int_rep.shape)
print("ir_int_rep.shape", ir_int_rep.shape)

## Concatenate the three weights into one big weight for TSNE

In [ ]:
all_rep = torch.cat((int_rep, rgb_int_rep, ir_int_rep))
print("all_rep.shape", all_rep.shape)

## Run TSNE on the concatenated weights
## Warning: takes a few minutes

In [ ]:
%%time
# This takes 10 minutes on entire dataset (12273)
emb_rep = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(all_rep)
print("emb_rep.shape", emb_rep.shape)

In [ ]:
plt.scatter(emb_rep[:, 0], emb_rep[:, 1], alpha=0.5, color='black', s=0.2)
plt.show()

## Display combined, RGB, and IR separately

In [ ]:
com = emb_rep[0*n_images:1*n_images]
rgb = emb_rep[1*n_images:2*n_images]
ir  = emb_rep[2*n_images:3*n_images]
print("com.shape", com.shape)
print("rgb.shape", rgb.shape)
print("ir.shape", ir.shape)
plt.scatter(com[:,0], com[:,1], alpha=0.5, color="b", s=5, label=f"RGB + IR")
plt.scatter(rgb[:,0], rgb[:,1], alpha=0.5, color="r", s=5, label=f"RGB")
plt.scatter(ir[:,0],  ir[:,1],  alpha=0.5, color="g", s=5, label=f"IR")
plt.title("TSNE of internal representations")
plt.xlabel("Component 0")
plt.ylabel("Component 1")
plt.tick_params(top=True, right=True)
plt.legend()
plt.show()

## Add features based on the image filepaths

In [ ]:
output_path = folder / 'output.csv'
prefix, postfix = len("sage_mobotix_cam_"), len("20220415-000000")
df = pd.read_csv(output_path, header=None, names=['image_path', 'ignore_0', 'image_class', 'ignore_1'])
df["image_name"] = df["image_path"].apply(lambda x: os.path.basename(x))
df["image_date"] = df["image_name"].apply(lambda x: x[prefix:prefix+postfix])
print(df[["image_name", "image_class"]])

## Let's focus on the RGB images

In [ ]:
#x = com[:, 0]  # Get the first column of rgb (x-coordinates)
#y = com[:, 1]  # Get the second column of rgb (y-coordinates)
x = rgb[:, 0]  # Get the first column of rgb (x-coordinates)
y = rgb[:, 1]  # Get the second column of rgb (y-coordinates)
#x = ir[:, 0]  # Get the first column of rgb (x-coordinates)
#y = ir[:, 1]  # Get the second column of rgb (y-coordinates)

print("x.shape", x.shape)
print("y.shape", y.shape)

## Let's highlight the different classifications

In [ ]:
# Find the unique classes
classes = df['image_class'].unique()
print(f"Unique classes: {classes}")

In [ ]:
# Create a scatter plot
plt.figure(figsize=(8, 6))  # Set the figure size

# Create a different color for each class
fig = plt.figure()
colors = ['black', 'r']
for label in classes:
    indices = label == df['image_class']
    plt.scatter(x[indices], y[indices], c=colors[label], label=f'Class {label}', alpha=0.5, s=5)

# Add text to the plot
plt.title('Scatter Plot')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

## Let's choose an image at random which has fire detected (`class==1`)
## And let's compare with it's nearest neighbors in the embedded space

In [ ]:
n_closest = 4

fire_indices = df[df['image_class'] == 1].index
print(f"fire_indices: {fire_indices}")
print(f"fire_indices.shape: {fire_indices.shape}")
random_fire = random.choice(fire_indices)
v = np.array([x[random_fire], y[random_fire]])

# Step 1: Broadcast V to match the shape of C
v_broadcasted = np.tile(v, (rgb.shape[0], 1))

# Step 2: Calculate the Euclidean distances
distances = np.linalg.norm(rgb - v_broadcasted, axis=1)

# Step 3: Find the indices of the n_closest smallest distances
closest_indices = np.argsort(distances)[:n_closest]

print(closest_indices)

## Lets plot the images which are most similar in the internal representation

In [ ]:
# Create a figure with a grid of subplots
fig, axes = plt.subplots(figsize=(8, 3*nrows), nrows=n_closest)

for i, index in enumerate(closest_indices):
    print(f"{df['image_path'][index]} is class={df['image_class'][index]}")
    image_path = df['image_path'][index]
    image = mpimg.imread("." + image_path)
    title = f"{df['image_date'][index]}, Class={df['image_class'][index]}"
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title(title)

# Display the figure
plt.show()


## Let's highlight the nearest neighbors on the plot

In [ ]:
# Create a scatter plot with a different color for each class
fig = plt.figure()
for label in classes:
    indices = label == df['image_class']
    plt.scatter(x[indices], y[indices], c=colors[label], label=f'Class {label}', alpha=0.5, s=5)

# Highlight the closest points
plt.scatter(x[closest_indices], y[closest_indices], c='blue', alpha=0.5, s=15)

# Add text to the plot
plt.title('Scatter Plot')

plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

In [ ]:
fraction = df['image_class'].sum() / len(df['image_class'])
print(f"Fraction of fire detection: {fraction:.5f}")